In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from sklearn.neighbors import KernelDensity
import functools
import pandas as pd
import math
import random
import copy
import seaborn as sns
import itertools
from itertools import groupby
import separation_algorithm as sepa
#import functions_eye_tracker_project as funcs
import sklearn
import os
from sklearn.neighbors import KernelDensity

In [66]:
#The function T_matrix_eff generates a transition matrix from a 2-d time-series
#The function new_series generates a new 2-d time-series given a Markov transition matrix and the bins associated with said matrix.


def T_matrix_eff(incrementX,incrementY,nbins='auto',bandwidth='auto'):

    if bandwidth == 'auto':
        l_band_X = 1.06*np.std(incrementX)*(len(incrementX)**(-0.15))
        l_band_Y = 1.06*np.std(incrementY)*(len(incrementY)**(-0.15))
    if nbins == 'auto':
        N_bin= 28
        l_folga_x= int(N_bin*2*l_band_X/(max(incrementX) - min(incrementX)))+1
        l_folga_y= int(N_bin*2*l_band_Y/(max(incrementY) - min(incrementY)))+1
        
        dX = (max(incrementX) - min(incrementX))/N_bin
        dY = (max(incrementY) - min(incrementY))/N_bin
        
        folga_x = max(l_folga_x/dX,l_folga_y/dY)*dX
        folga_y = max(l_folga_x/dX,l_folga_y/dY)*dY
        
        folga_x = max(folga_x,folga_y)
        folga_y = max(folga_x,folga_y)
        print('extra bins =' + str( max(folga_x,folga_y)))
        
        bins_X = np.arange(min(incrementX)-folga_x*dX,max(incrementX)+(folga_x+1)*dX,dX)
        bins_Y = np.arange(min(incrementY)-folga_y*dY,max(incrementY)+(folga_y+1)*dY,dY)
        
    if len(bins_X) != len(bins_Y):
        print(len(bins_X))
        print(len(bins_Y))
        return('error, should have same number of bins in X and Y')
        
        
        
    Q_matrix = np.zeros((len(bins_X)**2,len(bins_Y)**2))
    incrementX_list_kernel = []
    incrementY_list_kernel = []
    incrementTOT_list_kernel = []
    for k in range(len(incrementX)):
        elementary_vec_x = np.zeros(len(bins_X))
        elementary_vec_y = np.zeros(len(bins_Y))
        for i in range(len(bins_X)-1):     
            elementary_vec_x[i] = scipy.stats.norm.cdf((-incrementX[k]+bins_X[i+1])/l_band_X) - scipy.stats.norm.cdf((-incrementX[k]+bins_X[i])/l_band_X)
            elementary_vec_y[i] = scipy.stats.norm.cdf((-incrementY[k]+bins_Y[i+1])/l_band_Y) - scipy.stats.norm.cdf((-incrementY[k]+bins_Y[i])/l_band_Y)
        #being at the state 0 means that it is between 0 and 1. This is important for the reconstruction
        elementary_vec_x[len(bins_X)-1] = 1 - scipy.stats.norm.cdf((-incrementX[k]+bins_X[len(bins_X)-1])/l_band_X)
        elementary_vec_y[len(bins_Y)-1] = 1 - scipy.stats.norm.cdf((-incrementY[k]+bins_Y[len(bins_Y)-1])/l_band_Y)

        if np.abs(np.sum(elementary_vec_x)-1) > 0.001:
            print('normalization_error in X')
            print(np.sum(elementary_vec_x))
        if np.abs(np.sum(elementary_vec_y)-1) > 0.001:
            print('normalization_error in Y')
            
        incrementX_list_kernel.append(np.array(elementary_vec_x))
        incrementY_list_kernel.append(np.array(elementary_vec_y))
        current_incrementTOT_list_kernel = np.ndarray.flatten(np.outer(elementary_vec_x,elementary_vec_y))
        incrementTOT_list_kernel.append(np.ndarray.flatten(np.outer(elementary_vec_x,elementary_vec_y)))
        
        if k == 0:
            previous_incrementTOT_list_kernel = current_incrementTOT_list_kernel
        else:
            Q_matrix = Q_matrix + np.outer(previous_incrementTOT_list_kernel,current_incrementTOT_list_kernel)
            previous_incrementTOT_list_kernel = current_incrementTOT_list_kernel
            
            
            
        print(k)
            
    T_matrix = Q_matrix / np.sum(Q_matrix,axis=1)[:, np.newaxis]
    
    return(T_matrix,bins_X,bins_Y,bandwidth)



In [5]:
def new_series(T_matrix,bins_X,bins_Y,initial_state_x,initial_state_y,len_series):
    if len(bins_X) != len(bins_Y):
        print('error, bin numbers should be the same in X and Y')
    N_bins = len(bins_X)
    bin_wid_x = bins_X[1] - bins_X[0]
    bin_wid_y = bins_Y[1] - bins_Y[0]
    states = []
    current_state = N_bins*np.where(bins_X>initial_state_x)[0][0] + np.where(bins_Y>initial_state_y)[0][0]
    cum_matrix = np.cumsum(T_matrix,axis=1)
    for i in range(len_series):

        cum_vector = cum_matrix[current_state]
        rand_n = np.random.rand()
        new_state = np.where(cum_vector>rand_n)[0][0]
        states.append(new_state)
    
    int_state_X = np.array(np.array(states)/N_bins).astype(int)
    int_state_Y = np.array(np.array(states)%N_bins).astype(int)
    state_X = []
    state_Y = []
    for i in range(len(int_state_X)):
        state_X.append(bins_X[int(int_state_X[i])]+bin_wid_x*np.random.rand())
        state_Y.append(bins_Y[int(int_state_Y[i])]+bin_wid_y*np.random.rand())

    return(state_X,state_Y)
    
    